In [1]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

/Users/nascela/anaconda3/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [2]:
# mnist.DESCR

In [3]:
X, Y = mnist["data"], mnist["target"]
X.shape

(70000, 784)

In [5]:
Y

array(['5', '0', '4', ..., '4', '5', '6'], dtype=object)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap = "binary")
plt.axis("off")
plt.show()

In [ ]:
Y[0]

In [ ]:
import numpy as np
Y = Y.astype(np.uint8)

In [ ]:
Y[0]

In [ ]:
X_train, X_test, Y_train, Y_test = X[:60000], X[60000:], Y[:60000], Y[60000:]

In [ ]:
Y_train_5 = (Y_train == 5)
Y_test_5 = (Y_test == 5)

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, Y_train_5)

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits = 3, random_state = 42, shuffle=True)

for train_index, test_index in skfolds.split(X_train, Y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    Y_train_folds = Y_train_5[train_index]
    X_test_fold = X_train[test_index]
    Y_test_fold = Y_train_5[test_index]
    
    clone_clf.fit(X_train_folds, Y_train_folds)
    Y_pred =clone_clf.predict(X_test_fold)
    n_correct = sum(Y_pred == Y_test_fold)
    print(n_correct/len(Y_pred))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(sgd_clf, X_train, Y_train_5, cv = 3, scoring="accuracy")

In [ ]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, Y = None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype = bool)
    

In [ ]:
never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, Y_train_5, cv = 3, scoring = "accuracy")

In [ ]:
from sklearn.model_selection import cross_val_predict

Y_train_pred = cross_val_predict(sgd_clf,  X_train, Y_train_5, cv = 3)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_train_5, Y_train_pred)

In [ ]:
from sklearn.metrics import  precision_score, recall_score

precision_score(Y_train_5, Y_train_pred)


In [ ]:

recall_score(Y_train_5, Y_train_pred)

In [ ]:
from sklearn.metrics import  f1_score
f1_score(Y_train_5, Y_train_pred)

In [ ]:
Y_scores = sgd_clf.decision_function([some_digit])
Y_scores


In [ ]:
theshold = 0
Y_some_digit_pred = (Y_scores > theshold)
Y_some_digit_pred

In [ ]:
theshold = 8000
Y_some_digit_pred = (Y_scores > theshold)
Y_some_digit_pred

In [ ]:
Y_scores = cross_val_predict(sgd_clf, X_train, Y_train_5, cv=3, method="decision_function")

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(Y_train_5, Y_scores)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.legend(loc="center right", fontsize=16) # Not shown in the book
    plt.xlabel("Threshold", fontsize=16)        # Not shown
    plt.grid(True)                              # Not shown
    plt.axis([-50000, 50000, 0, 1])             # Not shown



recall_90_precision = recalls[np.argmax(precisions >= 0.90)]
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]


plt.figure(figsize=(8, 4))                                                                  # Not shown
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.plot([threshold_90_precision, threshold_90_precision], [0., 0.9], "r:")                 # Not shown
plt.plot([-50000, threshold_90_precision], [0.9, 0.9], "r:")                                # Not shown
plt.plot([-50000, threshold_90_precision], [recall_90_precision, recall_90_precision], "r:")# Not shown
plt.plot([threshold_90_precision], [0.9], "ro")                                             # Not shown
plt.plot([threshold_90_precision], [recall_90_precision], "ro")                             # Not shown
# save_fig("precision_recall_vs_threshold_plot")                                              # Not shown
plt.show()

In [ ]:
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]
Y_train_pred_90 = (Y_scores >= threshold_90_precision)

In [ ]:
precision_score(Y_train_5, Y_train_pred_90)

In [ ]:
recall_score(Y_train_5, Y_train_pred_90)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(Y_train_5, Y_scores)


In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
    plt.axis([0, 1, 0, 1])                                    # Not shown in the book
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) # Not shown
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)    # Not shown
    plt.grid(True)                                            # Not shown

plt.figure(figsize=(8, 6))                                    # Not shown
plot_roc_curve(fpr, tpr)
fpr_90 = fpr[np.argmax(tpr >= recall_90_precision)]           # Not shown
plt.plot([fpr_90, fpr_90], [0., recall_90_precision], "r:")   # Not shown
plt.plot([0.0, fpr_90], [recall_90_precision, recall_90_precision], "r:")  # Not shown
plt.plot([fpr_90], [recall_90_precision], "ro")               # Not shown
# save_fig("roc_curve_plot")                                    # Not shown
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_train_5, Y_scores)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
y_probas_forest = cross_val_predict(forest_clf, X_train, Y_train_5, cv = 3, method = "predict_proba")


In [ ]:
Y_scores_forest = y_probas_forest[:, 1]
fpr_forest, tpr_forest, thresholds_forest = roc_curve(Y_train_5, Y_scores_forest)

In [ ]:
recall_for_forest = tpr_forest[np.argmax(fpr_forest >= fpr_90)]

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, "b:", linewidth=2, label="SGD")
plot_roc_curve(fpr_forest, tpr_forest, "Random Forest")
plt.plot([fpr_90, fpr_90], [0., recall_90_precision], "r:")
plt.plot([0.0, fpr_90], [recall_90_precision, recall_90_precision], "r:")
plt.plot([fpr_90], [recall_90_precision], "ro")
plt.plot([fpr_90, fpr_90], [0., recall_for_forest], "r:")
plt.plot([fpr_90], [recall_for_forest], "ro")
plt.grid(True)
plt.legend(loc="lower right", fontsize=16)
# save_fig("roc_curve_comparison_plot")
plt.show()

In [ ]:
roc_auc_score(Y_train_5, Y_scores_forest)

In [ ]:
Y_train_pred_forest = cross_val_predict(forest_clf, X_train, Y_train_5, cv=3)
# y_probas_forest = cross_val_predict(forest_clf, X_train, Y_train_5, cv = 3, method = "predict_proba")

precision_score(Y_train_5, Y_train_pred_forest)

In [ ]:
 recall_score(Y_train_5, Y_train_pred_forest)

In [ ]:
from sklearn.svm import SVC
svm_clf = SVC()
svm_clf.fit(X_train, Y_train)
svm_clf.predict([some_digit])


In [ ]:
some_digit_score =svm_clf.decision_function([some_digit])
some_digit_score

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
ovr_clf = OneVsRestClassifier(SVC())
ovr_clf.fit(X_train, Y_train)
ovr_clf.predict([some_digit])

In [ ]:
sgd_clf.fit(X_train, Y_train)

In [ ]:
sgd_clf.predict([some_digit])

In [ ]:
sgd_clf.decision_function([some_digit])

In [ ]:
cross_val_score(sgd_clf, X_train, Y_train, cv = 3, scoring="accuracy")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))
cross_val_score(sgd_clf, X_train_scaled, Y_train, cv = 3, scoring="accuracy")

In [ ]:
Y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, Y_train, cv=3)
conf_mx = confusion_matrix(Y_train, Y_train_pred)
conf_mx

In [ ]:
plt.matshow(conf_mx, cmap = plt.cm.gray)
plt.show()

In [ ]:
row_sums = conf_mx.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx/row_sums
np.fill_diagonal(norm_conf_mx, 0)
plt.matshow(norm_conf_mx, cmap=plt.cm.gray)
plt.show()

In [ ]:
def plot_digit(data):
    image = data.reshape(28, 28)
    plt.imshow(image, cmap = mpl.cm.binary,
               interpolation="nearest")
    plt.axis("off")

    
    
def plot_digits(instances, images_per_row=10, **options):
    size = 28
    images_per_row = min(len(instances), images_per_row)
    # This is equivalent to n_rows = ceil(len(instances) / images_per_row):
    n_rows = (len(instances) - 1) // images_per_row + 1

    # Append empty images to fill the end of the grid, if needed:
    n_empty = n_rows * images_per_row - len(instances)
    padded_instances = np.concatenate([instances, np.zeros((n_empty, size * size))], axis=0)

    # Reshape the array so it's organized as a grid containing 28×28 images:
    image_grid = padded_instances.reshape((n_rows, images_per_row, size, size))

    # Combine axes 0 and 2 (vertical image grid axis, and vertical image axis),
    # and axes 1 and 3 (horizontal axes). We first need to move the axes that we
    # want to combine next to each other, using transpose(), and only then we
    # can reshape:
    big_image = image_grid.transpose(0, 2, 1, 3).reshape(n_rows * size,
                                                         images_per_row * size)
    # Now that we have a big image, we just need to show it:
    plt.imshow(big_image, cmap = mpl.cm.binary, **options)
    plt.axis("off")

In [ ]:
cl_a, cl_b = 3, 5
X_aa = X_train[(Y_train == cl_a) & (Y_train_pred == cl_a)]
X_ab = X_train[(Y_train == cl_a) & (Y_train_pred == cl_b)]
X_ba = X_train[(Y_train == cl_b) & (Y_train_pred == cl_a)]
X_bb = X_train[(Y_train == cl_b) & (Y_train_pred == cl_b)]

plt.figure(figsize=(8,8))
plt.subplot(221); plot_digits(X_aa[:25], images_per_row=5)
plt.subplot(222); plot_digits(X_ab[:25], images_per_row=5)
plt.subplot(223); plot_digits(X_ba[:25], images_per_row=5)
plt.subplot(224); plot_digits(X_bb[:25], images_per_row=5)
# save_fig("error_analysis_digits_plot")
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

Y_train_large = (Y_train >= 7)
Y_train_odd = (Y_train % 2 == 1)
Y_multilabel = np.c_[Y_train_large, Y_train_odd]

knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train, Y_multilabel)

In [ ]:
knn_clf.predict([some_digit])

In [ ]:
Y_train_knn_pred = cross_val_predict(knn_clf, X_train, Y_multilabel, cv = 3)
f1_score(Y_multilabel, Y_train_knn_pred, average="macro")

In [ ]:
noise = np.random.randint(0, 100,(len(X_train), 784))
X_train_mod = X_train + noise
noise = np.random.randint(0, 100,(len(X_test), 784))
X_test_mod = X_test + noise
Y_train_mod = X_train
Y_test_mod = X_test

In [ ]:
some_index = 0
plt.subplot(121); plot_digit(X_test_mod[some_index])
plt.subplot(122); plot_digit(Y_test_mod[some_index])
# save_fig("noisy_digit_example_plot")
plt.show()

In [ ]:
knn_clf.fit(X_train_mod, Y_train_mod)
clean_digit =knn_clf.predict([X_test_mod[some_index]])
plot_digit(clean_digit)

In [ ]:
some_digit = X[0]
some_digit_image = some_digit.reshape(28, 28)

plt.imshow(some_digit_image, cmap = "binary")
plt.axis("off")
plt.show()

In [ ]:
plt.matshow(some_digit_image, cmap = "binary")

In [ ]:
some_digit = X[1]
some_digit_image = some_digit.reshape(28, 28)

plt.matshow(some_digit_image, cmap = "binary")

In [ ]:
Y[1]

In [ ]:
from scipy.ndimage.interpolation import shift

In [ ]:
def Shift_image(image, dx, dy):
    return shift(image, [-dy, dx], cval=0, mode="constant")
#     shifted_image = shifted_image.reshape([-1]).reshape(28,28)
#     shifted_image = shifted_image.reshape(28,28)

In [ ]:
some_digit = X[101]
reshaped_image = some_digit.reshape(28, 28)
shifted_image = Shift_image(reshaped_image, 0, 5)
plt.matshow(shifted_image, cmap = "binary")

In [ ]:
some_digit = X[102]
reshaped_image = some_digit.reshape(28, 28)
shifted_image = Shift_image(reshaped_image, 5, 5)
plt.matshow(shifted_image, cmap = "binary")


plt.subplot(121)
plt.title("Defolt", fontsize=14)
plt.imshow(reshaped_image, cmap = "binary")


plt.subplot(122)
plt.title("Shifted right and up", fontsize=14)
plt.imshow(shifted_image, cmap = "binary")

plt.show()

print(Y[102])

# Tinanic dataset

In [ ]:
import os
import urllib.request

TITANIC_PATH = os.path.join("datasets", "titanic")
DOWNLOAD_URL = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/titanic/"

def fetch_titanic_data(url=DOWNLOAD_URL, path=TITANIC_PATH):
    if not os.path.isdir(path):
        os.makedirs(path)
    for filename in ("train.csv", "test.csv"):
        filepath = os.path.join(path, filename)
        if not os.path.isfile(filepath):
            print("Downloading", filename)
            urllib.request.urlretrieve(url + filename, filepath)

fetch_titanic_data()
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
train_data = train_data.set_index("PassengerId")
test_data = test_data.set_index("PassengerId")

In [ ]:
train_data.info()

In [ ]:
train_data[train_data["Sex"]=="female"]["Age"].median()

In [ ]:
train_data[(train_data["Pclass"]==1) & (train_data["Sex"]=="female")]["Age"].median()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer


num_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])


cat_pipeline = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("cat_encoder", OneHotEncoder(sparse=False)),
    ])



num_attribs = ["Age", "SibSp", "Parch", "Fare"]
cat_attribs = ["Pclass", "Sex", "Embarked"]

preprocess_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

X_train = preprocess_pipeline.fit_transform(train_data[num_attribs + cat_attribs])


In [ ]:
X_train

In [ ]:
y_train = train_data["Survived"]

In [ ]:
y_train

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators = 100, random_state=42)
forest_clf.fit(X_train, y_train)

In [ ]:
X_test = preprocess_pipeline.transform(test_data[num_attribs + cat_attribs])
y_pred =forest_clf.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_clf, X_train, y_train, cv = 10)
forest_scores.mean()

In [ ]:
from sklearn.svm import SVC

svm_clf = SVC(gamma="auto")
svm_scores = cross_val_score(svm_clf, X_train, y_train, cv=10)
svm_scores.mean()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot([1]*10, svm_scores, ".")
plt.plot([2]*10, forest_scores, ".")
plt.boxplot([svm_scores, forest_scores], labels=("SVM","Random Forest"))
plt.ylabel("Accuracy", fontsize=14)
plt.show()